# quantum_machine_learning.myQML.QSVM

**class quantum_machine_learning.myQML.QSVM(n_qubits, device=, kernel_circuit_label, angle_encoding_type='x', custom_kernel=None)**    

Class to solve an Supervised Machine Learning image mult-class classification problem using a Quantum-Classical Convolutional Neural Network. It defines a quantum convolutional layer and then a classical convolutional neural network. First, using random quantum variational angles, it processes the images and trains the classical network to find the optimal classical weights, and then, with these fixed weights, it trains the quantum layer. This algorithm is inspired by the paper *Quanvolutional Neural Networks: Powering image Recognition with Quantum Circuits* by Henderson et al. However, in the paper the angles of the quantum convolutional layer are fixed, while here we optimize them too. 

**Parameters:**
- **n_qubits** (int): Number of qubits used for the quantum circuit, which depends of the size of the quantum kernel used to process the image.
- **device** (str): Indicates whether the quantum circuits are executed locally (myQLM) or sent to a Qaptiva emulator as batches.
- **n_blocks** (int): Number of blocks in the variational quantum layer, where each of the blocks contains rotational gates and CNOTs (or other entangling two-qubit gates).
- **n_shots** (int): Number of executions of the corresponding quantum circuit to estimate a probability or the expectation value of an observable. If n_shots = None, the estimation of the quantum simulator has no shot noise.
- **optimizer_name** (str): Label of the optimizer used for the training of the classical neural network. Only 'Adam' optimizer is implemented currently.
- **loss** (str): Label of the loss function used for the training of the classical neural network, e.g. 'sparse_categorical_crossentropy'. Any loss function label already implemented in tensorflow keras can be used.
- **learning_rate** (str): Size of the step in the Adam optimizer. Not needed if a different optimizer is used.
- **opt_model_path** (str): Path where the optimal classical convolutional neural network is going to be saved.
- **np_arrays_path** (str): Path where the images after the quantum processing are saved, as well as the quantum random angles used for that processing.


**quantum_conv_kernel_circuit(x)**

This method implements a quantum convolutional kernel circuit for a custom QCCNN. It first applies an angle encoding feature map to map the input data (portion of the image) to a quantum state, and then applies series of rotation gates with random angles and CNOT gates to the qubits.

**Parameters:**
- **x** (numpy.ndarray): Input data vector of length n_qubits that is used as input angles of the rotation gates applied to each qubit in the feature map. 

**Returns**: An array of n_qubits elements with the Z expectation value of each of the qubits.

**Return type**:  numpy.ndarray


**quantum_conv_layer(image)**

This method slides the quantum kernel circuit through the whole images and returns the corresponding output image.


**Parameters:**
- **image** (numpy.ndarray): Rank 2 or 3 tensor with the pixel values of the image (with possibly more than one channel). 

**Returns:** The resulting image or tensor of usually rank 3.

**Return type:** numpy.ndarray 


**quantum_conv_preprocessing(train_images, test_images=None, save=True, params=None)**

This method calls quantum_conv_layer for all the images in the training and test sets, and stack the resulting tensor forming a rank 4 tensor for each set.

**Parameters:**
- **train_images** (numpy.ndarray): Images (with possibly multiple channels) in the training set used to optimize the network.
- **test_images** (numpy.ndarray): Images (with possibly multiple channels) in the test set used to assess the performance of the network.
- **save** (bool): Determines whether the train and test output quantum images are saved.
- **params** (numpy.ndarray): Array of the random angles of the variational ansatz of the quantum convolutional layer. If None, a new array of andom angle is created.

**Returns:** The output tensors resulting from the quantum preprocessing of the training and test sets, i.e., the quantum_train_images and quantum_test_images. It also returns the random angles employed.

**Return type:** quantum_train_images - numpy.ndarray (4D), quantum_test_images - numpy.ndarray (4D), params - numpy.ndarray (1D)


**classical_model()**

Defines a classical CNN model (using tensorflow keras) with custom hidden layers with the desired filters, kernel size and ativation function, as well as the output layer with the dsired number of classes and activation function. Additionally, it specifies the optimizer, loss function and metrics (such as the accuracy).

**Returns:** 

**Return type:**


**train(preprocessing, train_images, train_labels, validation_images, validation_labels, batch_size, n_epochs, early_stop_crit=False, patience=None, min_delta=None)**



**Parameters:**
- **preprocessing**: 
- **train_images**:
- **train_labels**:
- **validation_images**:
- **validation_labels**:
- **batch_size**:
- **n_epochs**:
- **early_stop_crit**:
- **patience**:
- **min_delta**: 

**Returns:** 

**Return type:**


**plot_quantum_images(train_images, quantum_train_images, n_samples)**



**Parameters:**
- **train_images**: 
- **quantum_train_images**:
- **n_samples**:

**Returns:** 

**Return type:**


**plot_loss(c_history=None, fig_path=None, fig_name='qccnn_loss.png')**



**Parameters:**
- **c_history**: 
- **fig_path**:
- **fig_name**:

**Returns:** 

**Return type:**



**optimize_quantum_params(train_images, train_labels, method='cobyla', max_iter=10, n_init=5, n_iters=20)**



**Parameters:**
- **train_images**: 
- **train_labels**:
- **method**:
- **max_iter**:
- **n_init**:
- **n_iters**:

**Returns:** 

**Return type:**


**predict(preprocessing, test_images, test_labels)**



**Parameters:**
- **preprocessing**: 
- **test_images**:
- **test_labels**:

**Returns:** 

**Return type:**


**plot_q_train_loss(train_accuracies, train_losses, fig_path=None, fig_name='qccnn_quantum_loss.png')**



**Parameters:**
- **train_accuracies**: 
- **train_losses**:
- **fig_path**:
- **fig_name**:

**Returns:** 

**Return type:**



